# В данном примере я описал регрессионную модель для заполнения пропущенных значений и указал промежуточную визуализацию через каждые 10 тыс заполнений. Проблема щас во времени обработки. Ждал 3 часа, но не смог заполнить даже 10 тыс

In [26]:

import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Загрузка данных
df = pd.read_csv('result/filled_P_l.csv')

# Преобразование столбца времени и установка его в качестве индекса
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

# Проверка на пропущенные значения
print(f"Total missing values in 'P_l': {df['P_l'].isnull().sum()}")

# Установка частоты данных, так как было предупреждение о ее отсутствии
df = df.asfreq('5T')

# Счетчик для отслеживания числа заполненных значений
filled_counter = 0

# Заполнение пропущенных значений с использованием ARIMA
for idx, row in df[df['P_l'].isnull()].iterrows():
    train_data = df.loc[:idx - pd.Timedelta(minutes=5), 'P_l'].dropna()
    model = ARIMA(train_data, order=(5,1,2))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    df.loc[idx, 'P_l'] = forecast[0]
    
    # Увеличиваем счетчик
    filled_counter += 1
    
    # Визуализация промежуточного результата каждые 1000 заполненных значений
    if filled_counter % 10000 == 0:
        print(f"Filled {filled_counter} missing values")
        plt.clf()  # Очистка текущего рисунка
        plt.plot(df['P_l'], label='P_l')
        plt.scatter([idx], [forecast[0]], color='red')  # Помечаем прогнозируемое значение
        plt.title(f'Electricity Consumption Over Time\nFilled Values: {filled_counter}')
        plt.xlabel('Time')
        plt.ylabel('P_l')
        plt.legend()
        plt.pause(0.05)  # Пауза для обновления графика

plt.show()

# Визуализация
plt.plot(df['P_l'], label='P_l')
plt.title('Electricity Consumption Over Time')
plt.xlabel('Time')
plt.ylabel('P_l')
plt.legend()
plt.show()




Total missing values in 'P_l': 141185


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/anaconda3/lib/python3.9/s

KeyboardInterrupt: 